## 6. Metaclasses and Attributes

### 47 Use `__getattr__`, `__getattribute__`, and `__setattr__` for Lazy Attributes

In [2]:
import logging

In [3]:
class LazyRecord:
    def __init__(self):
        self.exists = 5

    def __getattr__(self, name):
        value = f'Value for {name}'
        setattr(self, name, value)
        return value

In [4]:
data = LazyRecord()
print('Before:', data.__dict__)
print('foo:   ', data.foo)
print('After: ', data.__dict__)

Before: {'exists': 5}
foo:    Value for foo
After:  {'exists': 5, 'foo': 'Value for foo'}


In [5]:
class LoggingLazyRecord(LazyRecord):
    def __getattr__(self, name):
        print(f'* Called __getattr__({name!r}), '
              f'populating instance dictionary')
        result = super().__getattr__(name)
        print(f'* Returning {result!r}')
        return result

In [6]:
data = LoggingLazyRecord()
print('exists:     ', data.exists)
print('First foo:  ', data.foo)
print('Second foo: ', data.foo)

exists:      5
* Called __getattr__('foo'), populating instance dictionary
* Returning 'Value for foo'
First foo:   Value for foo
Second foo:  Value for foo


In [7]:
class ValidatingRecord:
    def __init__(self):
        self.exists = 5

    def __getattribute__(self, name):
        print(f'* Called __getattribute__({name!r})')
        try:
            value = super().__getattribute__(name)
            print(f'* Found {name!r}, returning {value!r}')
            return value
        except AttributeError:
            value = f'Value for {name}'
            print(f'* Setting {name!r} to {value!r}')
            setattr(self, name, value)
            return value

In [8]:
data = ValidatingRecord()
print('exists:     ', data.exists)
print('First foo:  ', data.foo)
print('Second foo: ', data.foo)

* Called __getattribute__('exists')
* Found 'exists', returning 5
exists:      5
* Called __getattribute__('foo')
* Setting 'foo' to 'Value for foo'
First foo:   Value for foo
* Called __getattribute__('foo')
* Found 'foo', returning 'Value for foo'
Second foo:  Value for foo


In [9]:
class MissingPropertyRecord:
    def __getattr__(self, name):
        if name == 'bad_name':
            raise AttributeError(f'{name} is missing')
        value = f'Value for {name}'
        setattr(self, name, value)
        return value

try:
    data = MissingPropertyRecord()
    assert data.foo == 'Value for foo'  # Test this works
    data.bad_name
except:
    logging.exception('Expected')
else:
    assert False

ERROR:root:Expected
Traceback (most recent call last):
  File "<ipython-input-9-7aefe36601dc>", line 12, in <module>
    data.bad_name
  File "<ipython-input-9-7aefe36601dc>", line 4, in __getattr__
    raise AttributeError(f'{name} is missing')
AttributeError: bad_name is missing


In [10]:
data = LoggingLazyRecord()  # Implements __getattr__
print('Before:         ', data.__dict__)
print('Has first foo:  ', hasattr(data, 'foo'))
print('After:          ', data.__dict__)
print('Has second foo: ', hasattr(data, 'foo'))

Before:          {'exists': 5}
* Called __getattr__('foo'), populating instance dictionary
* Returning 'Value for foo'
Has first foo:   True
After:           {'exists': 5, 'foo': 'Value for foo'}
Has second foo:  True


In [11]:
data = ValidatingRecord()  # Implements __getattribute__
print('Has first foo:  ', hasattr(data, 'foo'))
print('Has second foo: ', hasattr(data, 'foo'))

* Called __getattribute__('foo')
* Setting 'foo' to 'Value for foo'
Has first foo:   True
* Called __getattribute__('foo')
* Found 'foo', returning 'Value for foo'
Has second foo:  True


In [12]:
class SavingRecord:
    def __setattr__(self, name, value):
        # Save some data for the record
        pass
        super().__setattr__(name, value)

In [13]:
class LoggingSavingRecord(SavingRecord):
    def __setattr__(self, name, value):
        print(f'* Called __setattr__({name!r}, {value!r})')
        super().__setattr__(name, value)

data = LoggingSavingRecord()
print('Before: ', data.__dict__)
data.foo = 5
print('After:  ', data.__dict__)
data.foo = 7
print('Finally:', data.__dict__)

Before:  {}
* Called __setattr__('foo', 5)
After:   {'foo': 5}
* Called __setattr__('foo', 7)
Finally: {'foo': 7}


```python
# ex11.py

class BrokenDictionaryRecord:
    def __init__(self, data):
        self._data = {}

    def __getattribute__(self, name):
        print(f'* Called __getattribute__({name!r})')
        return self._data[name]

data = BrokenDictionaryRecord({'foo': 3})
data.foo
```

```shell
$ python item_47_ex_11.py 
* Called __getattribute__('foo')
* Called __getattribute__('_data')
... (993)
* Called __getattribute__('_data')
Traceback (most recent call last):
  File "item_47_ex_11.py", line 12, in <module>
    data.foo
  File "item_47_ex_11.py", line 9, in __getattribute__
    return self._data[name]
  File "item_47_ex_11.py", line 9, in __getattribute__
    return self._data[name]
  File "item_47_ex_11.py", line 9, in __getattribute__
    return self._data[name]
  [Previous line repeated 992 more times]
  File "item_47_ex_11.py", line 8, in __getattribute__
    print(f'* Called __getattribute__({name!r})')
RecursionError: maximum recursion depth exceeded while calling a Python object
```

In [16]:
class DictionaryRecord:
    def __init__(self, data):
        self._data = data

    def __getattribute__(self, name):
        # Prevent weird interactions with isinstance() used
        # by example code harness.
        if name == '__class__':
            return DictionaryRecord
        print(f'* Called __getattribute__({name!r})')
        data_dict = super().__getattribute__('_data')
        return data_dict[name]

data = DictionaryRecord({'foo': 3})
print('foo: ', data.foo)

* Called __getattribute__('foo')
foo:  3


> - `__getattr__`과 `__setattr__`을 사용해 객체의 애트리뷰트를 지연해 가져오거나 저장할 수 있다.
> - `__getattr__`은 애트리뷰트가 존재하지 않을 때만 호출되지만, `__getattribute__`는 애트리뷰트를 읽을 때마다 항상 호출된다는 점을 이해하라.
> - `__getattribute__`와 `__setattr__`에서 무한 재귀를 피하려면 `super()`에 있는(즉, `object` 클래스에 있는) 메서드를 사용해 인스턴스 애트리뷰트에 접근하라.